In [4]:
import codecs
import glob
import math
import os
from decimal import Decimal
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline


plt.rcParams["font.family"] = "Times New Roman"

data_dir = "data/input/励起状態/beta/NH/S1/dh6/*.out"
result_file = data_dir.split('励起状態')[1].split('*')[0].replace('/', '_')[1:-1]
result_dir = "data/output/" + result_file + ".csv"
paths = [p for p in glob.glob(data_dir)]

excited_state_1 = "Excited State   1"

gs_high_model_key = "RB3LYP"
initial_key_1 = "Excited State   1"
end_key_1 = "SavETr"
initial_key_2 = "low   system:  model energy"
end_key_2 = "ONIOM: calculating electric field derivatives."
result_dir = "data/output/result.csv"
df_output = pd.DataFrame()

for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_key_1を含む行を探す
        initial_key_1_df = df[df['Col_1'].str.contains(initial_key_1, case=False)]
        initial_key_1_index = initial_key_1_df.index[0]
        df_1 = df.iloc[initial_key_1_index:]
        # initial_key_1を含む行以降のend_key_1を含む行を抜き出す
        end_key_1_df = df_1[df_1['Col_1'].str.contains(end_key_1)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_1_index = end_key_1_df.index[0] - initial_key_1_index
        df_1 = df_1.iloc[:end_key_1_index]
        # 該当行の抜き出しが完了
        
        S1_data = [a for a in df_1[df_1['Col_1'].str.contains(excited_state_1)]['Col_1'].iloc[0].split(' ') if a != '']
        f_1 = float(S1_data[-2].split('f=')[-1])
        E_1 = float(S1_data[4])
        nm_1 = float(S1_data[-4])
    
        data_list = [f_1, E_1, nm_1]
        df_parameters = pd.DataFrame(data=data_list, 
                            columns=[torsion_angle],
                            index=["f_1", "S1", "nm_1"]).T
        
        
        
        gs_high_model = float([a for a in df[df['Col_1'].str.contains("RB3LYP")]['Col_1'].iloc[0].split(' ') if a != ''][4])
        initial_key_2_df = df[df['Col_1'].str.contains(initial_key_2, case=False)]
        initial_key_2_index = initial_key_2_df.index[0]
        df_2 = df.iloc[initial_key_2_index:]
        # initial_key_2を含む行以降のend_keyを含む行を抜き出す
        end_key_2_df = df_2[df_2['Col_1'].str.contains(end_key_2)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_2_index = end_key_2_df.index[0] - initial_key_2_index
        df_2 = df_2.iloc[:end_key_2_index]
        # 該当行の抜き出しが完了
        
        # 各エネルギー値の抜き出し
        low_model = float(list(df_2['Col_1'])[0].split(":")[-1])
        high_model = float(list(df_2['Col_1'])[1].split(":")[-1])
        low_real = float(list(df_2['Col_1'])[2].split(":")[-1])
        high_real = float(list(df_2['Col_1'])[3].split("=")[-1])
        
        S1_high_real = gs_high_model+(E_1/27.2116) + low_real - low_model
        gs_high_real = gs_high_model + low_real - low_model
        
        df_E = pd.DataFrame(data=[S1_high_real, gs_high_real, high_real, low_real, high_model, low_model], 
                            columns=[torsion_angle],
                            index=["S1_high_real", "gs_high_real", "high real", "low real", "high model", "low model"]).T
        df_result = pd.concat([df_parameters, df_E], axis=1)
        
        df_output = pd.concat([df_output, df_result]).sort_index()

In [ ]:
df_output

In [6]:
df_output.to_csv("data/output/" + result_file + "_all_data.csv")